# Training

In [23]:
from keras.preprocessing.image import ImageDataGenerator

from keras_resnet.models import ResNet50

from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.models.resnet import WEIGHTS_PATH_NO_TOP_50
from keras_retinanet import losses
from keras_retinanet.models.retinanet import retinanet_bbox

In [24]:
batch_size = 25
epochs = 200

In [25]:
def create_generator():
    # horizontal flip for preprocessing augmentation
    train_generator = ImageDataGenerator(horizontal_flip=True)
    # wrap in CSVGenerators
    csv_train_generator = CSVGenerator('./train.csv', './classes.csv', 
                                       train_generator, batch_size=batch_size)
    # no flip for val
    val_generator = ImageDataGenerator()
    csv_val_generator = CSVGenerator('./train.csv', './classes.csv', 
                                       val_generator, batch_size=batch_size)
    return csv_train_generator, csv_val_generator

In [26]:
def create_model(num_classes):
    input_shape = keras.layers.Input((None, None, 3))
    
    # first create the backbone ResNet50 model
    weights_path = keras.applications.imagenet_utils.get_file('ResNet-50-model.keras.h5', WEIGHTS_PATH_NO_TOP_50, 
                                                              cache_subdir='models', md5_hash='3e9f4e4f77bbe2c9bec13b53ee1c2319')
    backbone_model = ResNet50(input_shape, include_top=False, freeze_bn=True)
    # add feature pyramid network and subnets for box regression and object classification
    model = retinanet_bbox(inputs=input_shape, num_classes=num_classes, backbone=backbone_model)
    model.load_weights(weights_path, by_name=True)
    # compile model
    model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=1e-5, clipnorm=0.001)
    )
    
    return model

In [27]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [28]:
def create_callbacks():
    checkpoint = keras.callbacks.ModelCheckpoint('resnet50_csv_best.h5', verbose=1)
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)
    loss_history = LossHistory()
    
    return [checkpoint, lr_scheduler, loss_history]

In [ ]:
train_generator, val_generator = create_generator()
model = create_model(train_generator.num_classes())
print(model.summary())

callbacks = create_callbacks()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
padding_conv1 (ZeroPadding2D)   (None, None, None, 3 0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9408        padding_conv1[0][0]              
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
conv1_relu

/home/appsyoon/miniconda3/envs/capstone/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Output "nms" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "nms" during training.


In [ ]:
history = model.fit_generator(generator=train_generator, steps_per_epoch=train_generator.size()//batch_size, epochs=epochs, 
                              validation_data=val_generator, validation_steps=val_generator.size()//batch_size, 
                              callbacks=callbacks, verbose=1)

Epoch 1/200


/home/appsyoon/.local/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 2163 (shape (480, 640, 3)) contains the following invalid boxes: [array([ 570.,  304.,  642.,  368.,    3.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]
/home/appsyoon/.local/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 3426 (shape (544, 960, 3)) contains the following invalid boxes: [array([ 394.,  456.,  519.,  547.,    1.])].
  [annotations[invalid_index, :] for invalid_index in invalid_indices]
/home/appsyoon/.local/lib/python3.6/site-packages/keras_retinanet-0.0.1-py3.6.egg/keras_retinanet/preprocessing/generator.py:100: UserWarning: Image with id 2110 (shape (544, 960, 3)) contains the following invalid boxes: [array([ 464.,  423.,  578.,  548.,    1.])].
  [annotations[invalid_index, :] for invalid_index in 

In [ ]:
# save the history as a json for chart
import pickle

with open('history.pkl' , 'wb') as history_file:
    pickle.dump(history.history, history_file)

In [ ]:
with open('history.pkl', 'rb') as history_file:
    data = pickle.load(history_file)
    print(data)